#NPL Fake News DataSet#

Lucas Jaenisch Lopes

In [1]:
!pip install tomotopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Initial Imports#

In [2]:
import sys
import spacy
import pandas as pd
import tomotopy as tp
import numpy as np
import gensim  
import re
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive

spacy.cli.download("en_core_web_md")
nlp = spacy.load('en_core_web_md')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


#Load Documents#

In [3]:
drive.mount('/content/drive')
fpath='/content/drive/MyDrive/'
df=pd.read_csv(fpath+'Fake.csv', nrows = 2000)
df.columns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['title', 'text', 'subject', 'date'], dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2000 non-null   object
 1   text     2000 non-null   object
 2   subject  2000 non-null   object
 3   date     2000 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB


In [5]:
df.drop('subject', inplace=True, axis=1)
df.drop('date', inplace=True, axis=1)
df.drop('title', inplace=True, axis=1)

#NLP#

In [6]:
def get_average_feature_length(docs, feature):
  total_size = 0
  max_size = 0
  min_size = 999

  # print(docs)
  # print(feature)

  for i in docs:
    length = len(i)
    total_size += length
  
    if length > max_size:
      max_size = length

    if length < min_size:
      min_size = length

  print(f"\n{'-'*15} {feature} {'-'*15}")
  print(f"Average length: {int(total_size / len(docs))}")
  print(f"Max length: {max_size}")
  print(f"Min length: {min_size}")
  print(f"Total length (not unique): {total_size}")
  return int(total_size / len(docs))

avg_text_size = get_average_feature_length(df['text'], 'text')


--------------- text ---------------
Average length: 2513
Max length: 8269
Min length: 359
Total length (not unique): 5027750


#HTML to plain text#


In [7]:
from bs4 import BeautifulSoup as soup

clean_text = []
r = len(df)

for f in df.columns:
  if f == 'text':
    for i in df[f][:r]:
      clean_text.append((soup(i, "lxml").text, len(i)))

In [8]:
# remove objects where body length is greater than the avg body length

new_text = []

for i in range(r):
  if not clean_text[i][1] > avg_text_size:
    new_text.append(clean_text[i][0])

In [9]:
df.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [10]:
# Rejoin all remaining rows to the DataFrame
clean_df = pd.DataFrame()
clean_df['text'] = new_text

df = clean_df

#Stop Words#

In [11]:
spacy.cli.download("en_core_web_md")
nlp = spacy.load('en_core_web_md')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
def convert_to_string(df, feature) -> str:
  output = []
  for i in range(len(df[feature])):
    output.append(df[feature][i].strip())
  return output

texts = convert_to_string(df, 'text')

dlemma = []
for i in texts:
  tdoc=nlp(i.lower())
  lm=" ".join([token.lemma_ for token in tdoc  if not(token.pos_ == 'NUM' or token.is_stop == True or token.is_digit == True or token.is_punct == True or token.lemma == False)])
  dlemma.append(lm)

for d1, d2 in zip(texts, dlemma):
  print(d1, '\n=>', d2)
  print()


#Bigrams#

In [ ]:
from gensim.models.phrases import Phrases, Phraser

import gensim.utils

In [ ]:
dtoken  = [gensim.utils.simple_preprocess(d, deacc= True, min_len=3) for d in dlemma]
phrases = Phrases(dtoken, min_count=6, threshold=15)
bigram  = Phraser(phrases)
bdocs   = [bigram[d] for d in dtoken]

[print(i) for i in bdocs]

In [ ]:
bigrams = []

for i in bdocs:
  for j in i:
    bigrams.append(j)
    if '_' in j:
      print(j)
bigramized = bigrams

In [ ]:
print(bigramized)

#Removing Dups#

In [ ]:
uniques = set(bigramized)
uniques = dict(zip(range(len(uniques)), uniques))

print("Total bigrams: ", len(bigramized))
print("Unique bigrams: ",  len(uniques))

In [ ]:
uniques.values()

#Tokenize#

In [ ]:
def tokenize(dlemma):
  max_size = len(dlemma); count = 0
  output = []
  
  for i in dlemma:
    if count % 200 == 0:
      print(f"{count}/{max_size}")
    
    aux = []; doc = nlp(i)

    for token in doc:
      if token.pos_ in ['NOUN', 'PROPN']:
        aux.append(token.lemma_)
    output.append(aux)
    count += 1
  return output
        
tokens = tokenize(uniques.values())
tokens[:10]

In [ ]:
# remove empty objects from the tokens

print("Old number of Tokens: ", len(tokens))
clean_tokens = []
for t in tokens:
  if t != []:
    clean_tokens.append(t)
tokens = clean_tokens
print("New number of Tokens: ", len(tokens))

In [ ]:
# [print(d) for d in tokens]

#Metrics#

In [ ]:
number_of_docs = len(tokens)
number_of_docs

In [ ]:
def stats_about_the_docs(docs):
  shortest_doc, largest_doc = "_"*100, ""
  max_size, min_size = 0, 999
  total_size = 0

  for i in docs:
      length = len(i[0])
      total_size += length

      if length > max_size:
        max_size = length
        largest_doc = i[0]

      if length < min_size:
        min_size = length
        shortest_doc = i[0]

  print(f"Average word length: {int(total_size / len(docs))}")
  print(f"\nMax length {max_size} : {largest_doc}")
  print(f"Min length  {min_size} : {shortest_doc}")

stats_about_the_docs(tokens)

In [ ]:
def create_single_string(tokens):
  output = ''
  for i in tokens:
    for j in i:
      output += j + " "

  return output

output = create_single_string(tokens)
output

#WordCloud#

In [ ]:
import wordcloud as wc
import matplotlib.pyplot as plt

mycloud = wc.WordCloud().generate(output)
plt.figure(figsize=(15,10))
plt.imshow(mycloud)

In [ ]:
[print(d) for d in bdocs]

In [ ]:
print(bdocs)
# print(collection)

#Dictionary#


In [ ]:
from gensim.corpora import Dictionary

In [ ]:
dictionary = Dictionary(bdocs)
dictionary.filter_extremes(keep_n = 10000, no_above= 0.8, no_below = 20) #==> the collection is too small for filtering it
for w in dictionary.values():
  print(dictionary.token2id[w],w) # show word id and value

In [ ]:
#build a bag-of-words - corpus
bagofwords = [dictionary.doc2bow(d) for d in bdocs]
[print(d) for d in bagofwords]

In [ ]:
#just to show the matrix of bag-of-words
uniquewords=set()
for d in bdocs:
  [uniquewords.add(w) for w in d]
#create a dictionary for every doc with 0
coldict=[]
for d in bdocs:
  coldict.append(dict.fromkeys(uniquewords,0))
for i,d in enumerate(bdocs):
  for w in d:
    coldict[i][w]+=1
columns=list(uniquewords)
bagDoc=pd.DataFrame(coldict,columns=columns)
bagDoc

In [ ]:
#just to show the tf-idf version
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
colAnt=[]
for d in bdocs:
  colAnt.append(' '.join(w for w in d))
X = vectorizer.fit_transform(colAnt)
features=vectorizer.get_feature_names()
dense = X.todense()
denseList=dense.tolist()
dfTfIdf = pd.DataFrame(denseList, columns=set(features))
dfTfIdf

In [ ]:
from gensim import models
from gensim.models import LdaModel
tfidf = models.TfidfModel(bagofwords)
corpus_tfidf = tfidf[bagofwords]
# myModel = LdaModel(corpus = corpus_tfidf,num_topics= 3, random_state= 27644437, id2word = dictionary, alpha = 'auto',per_word_topics = True,passes = 100)
# for t in myModel.show_topics():
#   print(t)
print('-----')
myModel = LdaModel(corpus = bagofwords,num_topics= 10, random_state= 27644437, id2word = dictionary, alpha = 'auto',per_word_topics = True,passes = 100)
for t in myModel.show_topics():
  print(t)

#Docs Topics#

In [ ]:
for i,d in enumerate(bagofwords): # collection and bagofwords must be synchronized
  print(bdocs[i],':',myModel.get_document_topics(d,minimum_probability=0.3)) #threshold


#Topic Metrics#

In [ ]:
from gensim.models import CoherenceModel  #c_v
metrics=['u_mass', 'c_v', 'c_uci', 'c_npmi']
for metric in metrics: #,'c_w2v']:
  myModelCoher = CoherenceModel(model=myModel, texts= bdocs, dictionary=dictionary, coherence=metric)  
  print('%-7s: %2.4f'%(metric,myModelCoher.get_coherence()))

In [ ]:
tops=[]
for k in range(10):
    t=[]
    for wt in myModel.show_topic(k):
      t.append(wt[0])
    tops.append(t)
for t in tops:
  topic_coher=CoherenceModel(topics=[t], texts= bdocs, dictionary=dictionary, coherence=metrics[1])
  print('topic:',t)
  print('C_v:%.3f'%(topic_coher.get_coherence()))
print("All:")
topic_coher=CoherenceModel(topics=tops, texts= bdocs, dictionary=dictionary, coherence=metrics[1])
print('C_v:%.3f'%(topic_coher.get_coherence()))
# top topics
print('Topics with the highest coherence score the coherence for each topic.')
myModel.top_topics(corpus=bagofwords,dictionary=dictionary,coherence=metrics[1],texts=dlemma,topn=5)

#Topics Similarity#

In [ ]:
#Build a dictionary mapping words to ids
#topics_list: list of topics with probabilites
#dart is the resulting dictionary
def getTopicsDict(topics_list,dart):
	sjm=set()
	[[sjm.add(w[0]) for w in t] for t in topics_list]
	nextid=len(dart)
	for k in sjm:
		try:
			dart[k]
		except:
			dart[k]=nextid
			nextid+=1
##
## based on the dictionary (topic_dict) create a new topic list with id
def getTopicsBow(topics_list,topic_dict):
	top_bow=[]
	for l in topics_list:
		ltup=[]
		for t in l:
			ltup.append((topic_dict[t[0]],t[1]))
		top_bow.append(ltup)
	return top_bow
#
def getSimTopics(topic,top_list_bow, dist):
	simTop={}
	for i,t in enumerate(top_list_bow):
		if dist==1:
			simTop[i]=hellinger(t,topic)
		if dist==2:
			simTop[i]=jaccard(topic,t) # returns an error :( - to check
	#sort dict
	ssimTop={k: v for k, v in sorted(simTop.items(), key=lambda item: item[1])}
	return ssimTop